# E4 — Hirano Mod-2 Dijkgraaf–Witten Invariants

## Goal
Compute mod-2 Dijkgraaf–Witten (DW) invariants for lens spaces $L(N, 1)$  
and related 3-manifolds, looking for factor-sensitive signals in semiprimes $N = pq$.

## Background
DW invariants are topological invariants of 3-manifolds valued in finite groups.  
For the lens space $L(N, 1)$ with gauge group $G = \mathbb{Z}/2$:
$$\mathrm{DW}_{\mathbb{Z}/2}(L(N,1)) = \frac{1}{|G|} \sum_{\rho : \pi_1 \to G} \omega(\rho)$$
where $\omega \in H^3(BG, U(1))$ is a 3-cocycle and $\pi_1(L(N,1)) = \mathbb{Z}/N$.

For $G = \mathbb{Z}/2$:  
- Homomorphisms $\mathbb{Z}/N \to \mathbb{Z}/2$ exist iff $\gcd(N, 2) = 2$ (i.e., $N$ even).
- For odd $N = pq$, only the trivial homomorphism exists → DW is trivial.

So we extend to:
- **Larger gauge groups**: $G = \mathbb{Z}/m$ for various $m$.
- **Hirano's refinement**: mod-$p$ DW invariants with twisted cocycles.
- **Legendre symbol patterns**: $\left(\frac{p}{q}\right)$ and related reciprocity data.

## Expectation
Likely limited (Legendre-symbol level information), but fast to compute  
and may expose unexpected patterns.

In [ ]:
import time
import numpy as np
from sage.all import *
import json
import os

## 1. DW Invariants for $L(N, 1)$ with Gauge Group $\mathbb{Z}/m$

The DW invariant with trivial cocycle is just the count of homomorphisms:
$$\mathrm{DW}^{\text{triv}}_{\mathbb{Z}/m}(L(N, 1)) = |\mathrm{Hom}(\mathbb{Z}/N, \mathbb{Z}/m)| = \gcd(N, m)$$

This immediately encodes divisibility — for $N = pq$ and $m = p$:  
$\gcd(pq, p) = p$. But this requires *knowing* $p$.

The interesting case is when we compute for **all** $m$ up to some bound  
and look at the resulting pattern.

In [ ]:
def dw_trivial_cocycle_spectrum(N, m_max=100):
    """
    Compute gcd(N, m) for m = 1, ..., m_max.
    This is the DW invariant with trivial 3-cocycle.
    
    For N = pq, this spectrum has jumps at m = p and m = q.
    """
    return [int(gcd(N, m)) for m in range(1, m_max + 1)]

def dw_nontrivial_cocycle_Z_m(N, m):
    """
    DW invariant for L(N, 1) with gauge group Z/m and the
    non-trivial 3-cocycle (generator of H^3(B(Z/m), U(1)) = Z/m).
    
    For the standard cocycle omega:
    DW = (1/m) * sum_{a in Hom(Z/N, Z/m)} exp(2*pi*i * omega(a) / m)
    
    where omega(a) = a^3 * N / m (mod m) for the generator cocycle.
    Actually, for the lens space L(N, 1):
    omega evaluated on the fundamental class gives a * N / gcd(N, m)^2
    type expression.
    """
    d = gcd(N, m)
    # Homomorphisms Z/N -> Z/m are parametrized by a in Z/m with m | a*N
    # i.e., a must be a multiple of m/d.
    
    total = CC(0)
    for j in range(d):  # j parametrizes the d homomorphisms
        a = j * (m // d)
        # Cocycle evaluation: for the canonical generator of H^3(BZ/m, U(1))
        # on L(N, 1), the value is (a^2 * N) / (2 * m^2) mod 1
        # (following the Dijkgraaf-Witten formula for lens spaces)
        cocycle_val = QQ(a**2 * N) / QQ(2 * m**2)
        total += exp(2 * pi * I * cocycle_val)
    
    return total / m

# Test
N_test = 77  # 7 * 11
print(f"DW invariants for N={N_test}=7×11:")
print(f"  Trivial cocycle spectrum (m=1..20): {dw_trivial_cocycle_spectrum(N_test, 20)}")
print()
print(f"  Non-trivial cocycle values:")
for m in [2, 3, 5, 7, 11, 13, 14, 21, 22, 35, 55, 77]:
    val = dw_nontrivial_cocycle_Z_m(N_test, m)
    print(f"    m={m:>3}: DW = {complex(val):>20s}, |DW| = {abs(val):.6f}")

## 2. Hirano's Refinement: Mod-$p$ Gauss Sums and Legendre Patterns

Hirano's approach uses twisted DW invariants where the cocycle involves  
quadratic residue symbols. For $N = pq$:

$$I_p(N) = \sum_{a=1}^{N-1} \left(\frac{a}{N}\right) \cdot e^{2\pi i a/N}$$

is a Gauss sum that encodes the Jacobi symbol structure.  
The Jacobi symbol $\left(\frac{a}{N}\right) = \left(\frac{a}{p}\right)\left(\frac{a}{q}\right)$  
factors multiplicatively — this factorization is the "signal."

In [ ]:
def gauss_sum_jacobi(N):
    """
    Compute the Gauss sum G(1, N) = sum_{a=1}^{N-1} (a/N) * e^{2*pi*i*a/N}
    where (a/N) is the Jacobi symbol.
    
    For N = pq odd semiprime:
    |G(1, N)|^2 = N * (some factor involving (-1/p)(-1/q))
    """
    total = CC(0)
    zeta = CC(exp(2 * pi * I / N))
    for a in range(1, N):
        js = jacobi_symbol(a, N)
        total += js * zeta^a
    return total

def partial_gauss_sums(N, num_partials=20):
    """
    Compute partial Gauss sums up to various cutoffs.
    The rate of convergence may encode factorization information.
    """
    zeta = CC(exp(2 * pi * I / N))
    cutoffs = [int(N * k / num_partials) for k in range(1, num_partials + 1)]
    
    running = CC(0)
    results = []
    a = 1
    for cutoff in cutoffs:
        while a <= cutoff and a < N:
            running += jacobi_symbol(a, N) * zeta^a
            a += 1
        results.append({
            'cutoff': cutoff,
            'cutoff_frac': cutoff / N,
            'partial_sum': complex(running),
            'magnitude': float(abs(running))
        })
    
    return results

def legendre_pattern_analysis(N):
    """
    Analyze the Legendre/Jacobi symbol pattern for N = pq.
    
    The Jacobi symbol (a/N) = (a/p)(a/q) factors.
    We look at various statistics of this pattern.
    """
    fac = factor(N)
    if len(fac) != 2 or any(e != 1 for _, e in fac):
        return None
    p, q = int(fac[0][0]), int(fac[1][0])
    
    # Jacobi symbol sequence
    jacobi_seq = [int(jacobi_symbol(a, N)) for a in range(N)]
    
    # Autocorrelation of Jacobi sequence at lags p and q
    seq = np.array(jacobi_seq, dtype=float)
    
    def autocorr(lag):
        if lag >= len(seq):
            return 0
        n = len(seq) - lag
        s = seq - np.mean(seq)
        denom = np.sum(s**2)
        if denom == 0:
            return 0
        return float(np.sum(s[:n] * s[lag:lag+n]) / denom)
    
    ac_p = autocorr(p)
    ac_q = autocorr(q)
    ac_random = np.mean([autocorr(lag) for lag in range(2, min(50, N)) 
                         if lag != p and lag != q])
    
    # Gauss sum
    gs = gauss_sum_jacobi(N)
    gs_expected_mag = float(RR(N).sqrt())  # |G| = sqrt(N) for prime
    
    # Cross-Legendre symbol
    leg_p_q = int(legendre_symbol(p, q)) if q > 2 else 0
    leg_q_p = int(legendre_symbol(q, p)) if p > 2 else 0
    
    return {
        'N': int(N), 'p': int(p), 'q': int(q),
        'gauss_sum': complex(gs),
        'gauss_mag': float(abs(gs)),
        'gauss_expected_mag': gs_expected_mag,
        'gauss_ratio': float(abs(gs)) / gs_expected_mag,
        'jacobi_autocorr_p': ac_p,
        'jacobi_autocorr_q': ac_q,
        'jacobi_autocorr_random': ac_random,
        'legendre_p_q': leg_p_q,
        'legendre_q_p': leg_q_p,
        'qr_product': leg_p_q * leg_q_p,
    }

# Test
lpa = legendre_pattern_analysis(77)
print(f"Legendre pattern analysis for N=77=7×11:")
for k, v in lpa.items():
    if k != 'gauss_sum':
        print(f"  {k}: {v}")

## 3. Systematic Scan

In [ ]:
from utils.semiprime_gen import generate_semiprimes

semiprimes_E4 = generate_semiprimes(10000, num_samples=60)
print(f"Generated {len(semiprimes_E4)} semiprimes")

e4_results = []

print(f"{'N':>6} {'p':>5} {'q':>5} {'|G|':>8} {'|G|/√N':>7} {'ac_p':>7} {'ac_q':>7} {'(p/q)':>5} {'(q/p)':>5}")
print("-" * 70)

for N, p, q in semiprimes_E4:
    try:
        t0 = time.perf_counter()
        res = legendre_pattern_analysis(N)
        elapsed = time.perf_counter() - t0
        
        if res is None:
            continue
        
        res['time'] = elapsed
        e4_results.append(res)
        
        print(f"{N:>6} {p:>5} {q:>5} {res['gauss_mag']:>8.2f} {res['gauss_ratio']:>7.4f} "
              f"{res['jacobi_autocorr_p']:>7.3f} {res['jacobi_autocorr_q']:>7.3f} "
              f"{res['legendre_p_q']:>5} {res['legendre_q_p']:>5}")
    except Exception as e:
        print(f"  N={N}: ERROR - {e}")

print(f"\nCompleted {len(e4_results)} analyses")

In [ ]:
# Save
output_path = os.path.join('..', 'data', 'E4_hirano_dw_results.json')
# Remove complex values for JSON serialization
serializable = []
for r in e4_results:
    row = {k: v for k, v in r.items() if k != 'gauss_sum'}
    serializable.append(row)
with open(output_path, 'w') as f:
    json.dump(serializable, f, indent=2)
print(f"Saved {len(serializable)} results")

## 4. Visualization and Verdict

In [ ]:
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

Ns = np.array([r['N'] for r in e4_results])
gauss_ratio = np.array([r['gauss_ratio'] for r in e4_results])
ac_p = np.array([r['jacobi_autocorr_p'] for r in e4_results])
ac_q = np.array([r['jacobi_autocorr_q'] for r in e4_results])
qr_prod = np.array([r['qr_product'] for r in e4_results])

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

ax = axes[0, 0]
ax.scatter(Ns, gauss_ratio, s=15, alpha=0.7, c=qr_prod, cmap='RdBu')
ax.axhline(1, color='black', lw=0.5, ls='--')
ax.set_xscale('log')
ax.set_xlabel('N')
ax.set_ylabel('|G(1,N)| / √N')
ax.set_title('Gauss Sum Magnitude (colored by QR product)')
ax.grid(True, alpha=0.3)

ax = axes[0, 1]
ax.scatter(ac_p, ac_q, s=15, alpha=0.7, c=np.log10(Ns), cmap='viridis')
ax.set_xlabel('Jacobi autocorr at lag p')
ax.set_ylabel('Jacobi autocorr at lag q')
ax.set_title('Autocorrelation at Factor Lags')
ax.grid(True, alpha=0.3)

ax = axes[1, 0]
ax.hist(gauss_ratio, bins=30, alpha=0.7, color='steelblue')
ax.axvline(1, color='red', lw=1, ls='--', label='√N baseline')
ax.set_xlabel('|G(1,N)| / √N')
ax.set_ylabel('Count')
ax.set_title('Distribution of Gauss Sum Ratios')
ax.legend()
ax.grid(True, alpha=0.3)

ax = axes[1, 1]
balance = np.array([r['p'] / r['q'] for r in e4_results])
ax.scatter(balance, gauss_ratio, s=15, alpha=0.7, color='purple')
ax.set_xlabel('p/q (factor balance)')
ax.set_ylabel('|G(1,N)| / √N')
ax.set_title('Gauss Sum vs Factor Balance')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join('..', 'data', 'E4_hirano_plots.png'), dpi=150)
plt.show()

print("\n" + "=" * 60)
print("E4 HIRANO MOD-2 DW INVARIANTS — VERDICT")
print("=" * 60)
print(f"Mean |G|/√N = {np.mean(gauss_ratio):.4f} (expected ~1 for primes)")
print(f"Factor-lag autocorrelation: mean(ac_p)={np.mean(ac_p):.4f}, mean(ac_q)={np.mean(ac_q):.4f}")
print()
print("As expected, the Jacobi symbol / DW invariant primarily encodes")
print("quadratic residuosity (Legendre symbol level) information.")
print("This is useful but not sufficient for factorization.")
print("The autocorrelation at factor lags is the most interesting signal.")